In [1]:
import numpy as np
import pandas as pd

In [25]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
X=[]
Y=[]
lines=[]


#data_path='/content/drive/My Drive/Self-driving-car-dataset/data.txt'
data_path='C:\\Users\\HP\\Desktop\\ML\\Self-Driving-Car\\driving_dataset\\data.txt'

#images_path='/content/drive/My Drive/Self-driving-car-dataset/images/'
images_path='C:\\Users\\HP\\Desktop\\ML\\Self-Driving-Car\\driving_dataset\\'

with open(data_path) as file:
    lines=file.readlines()
    

for line in lines:
    w=line.split()
    X.append(w[0])
    Y.append(float(w[1]))

In [9]:
for i,x in enumerate(X):
    X[i]=images_path+X[i]
    
X[4]

'C:\\Users\\HP\\Desktop\\ML\\Self-Driving-Car\\driving_dataset\\4.jpg'

In [10]:
import math

for i,y in enumerate(Y):
    Y[i]=(y*math.pi)/180
    
print(Y[89])

0.024609142453120045


In [11]:
import numpy as np
import cv2

def rotate_image(img, smoothed_angle, degrees):
    rows,cols,_ = img.shape
    if smoothed_angle!=degrees:
        smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    
    return dst, smoothed_angle

In [12]:
i=0
import os
#wheel_path='/content/drive/My Drive/Self-driving-car-dataset/wheel-4.jpg'
wheel_path='C:\\Users\\HP\\Desktop\\ML\\Self-Driving-Car\\wheel-4.jpg'
wheel=cv2.imread(wheel_path)
smoothed_angle=0
smoothed_angle_actual=0

wheel.shape

(240, 240, 3)

In [13]:
#model.save('m1.h5')
# from keras.models import load_model 
import tensorflow.compat.v1 as tf
model_path='C:\\Users\\HP\\Desktop\\ML\\Self-Driving-Car\\model.h5'
model = tf.keras.models.load_model(model_path,compile=False)

In [ ]:
i=0
for x in X:
    img=cv2.imread(x)
    cropped_img=cv2.resize(img[-150:],(200,66))/255.0
    cv2.imshow('frame',img)
    
    rad=model.predict(np.array([cropped_img]))[0][0]
    
    pred_deg=(rad*180)/(math.pi)
    w,smoothed_angle=rotate_image(wheel,smoothed_angle,pred_deg)
    cv2.imshow('Prediction',w)
    
    actual_deg=(Y[i]*180)/(math.pi)
    w2,smoothed_angle_actual=rotate_image(wheel,smoothed_angle_actual,actual_deg)
    cv2.imshow('Actual',w2)

    s="Prediction: "+str(pred_deg)+" Actual: "+str(actual_deg)+"\n"
    s=s.encode()
    os.write(1,s)
    
    i+=1;
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cv2.destroyAllWindows()